In [1]:
import sys

import numpy as np
from scipy import integrate
from struct import pack, unpack

sys.path.insert(0, "../")
from paths import FIG_DIR, DATA_DIR

In [2]:
def compute_lcs_volume(p, ssb):
    normal = ssb @ p
    if len(np.unique(normal)) < len(normal):
        return 1  # upper bound
    vertices_lcs = list(np.where(normal < 0)[0])
    normal_short = normal[vertices_lcs]
    with np.errstate(divide='ignore'):
        factors = normal_short[:, None]/(normal_short[:, None] - normal[None, :])
        factors = np.nan_to_num(factors, copy=False, nan=1.0, posinf=1.0, neginf=1.0)
    summands = np.prod(factors, axis=1)
    return np.sum(summands)

In [5]:
def f3(y, x, ssb_matrix):
    p = np.array([x, y, 1-x-y])
    return compute_lcs_volume(p, ssb_matrix)**2

ssb_tr = np.array([[0, 1, 1], [-1, 0, 1], [-1, -1, 0]])
ssb_intr = np.array([[0, 1, -1], [-1, 0, 1], [1, -1, 0]])

val, err = integrate.dblquad(f3, 0, 1, 0, lambda x: 1-x, args=(ssb_tr,))
print("Integral:", 2*val, "Prob. intr.:", 1-6*val, "Error:", 6*err)
val, err = integrate.dblquad(f3, 0, 1, 0, lambda x: 1-x, args=(ssb_intr,))
print("Integral:", 2*val, "Prob. intr.:", 1-6*val, "Error:", 6*err)

Integral: 0.3333333338408437 Prob. intr.: -1.5225309901722994e-09 Error: 8.663295278119672e-08
Integral: 0.2515928246444483 Prob. intr.: 0.24522152606665504 Error: 8.916985669204123e-08


In [2]:
def compute_lcs_volume_vectorized(p_matr, ssb):
    normal = - p_matr @ ssb
    normal_short = np.where(normal < 0, normal, 0)
    with np.errstate(divide='ignore'):
        factors = normal_short[:, :, None] / (normal_short[:, :, None] - normal[:, None, :])
        factors = np.nan_to_num(factors, copy=False, nan=1.0, posinf=1.0, neginf=1.0)
    summands = np.prod(factors, axis=2)
    return np.sum(summands, axis=1)

In [6]:
num_samples = 7.95 * 10**10
m = 3
rng = np.random.default_rng(2025)
ssb_intr = np.array([[0, 1, -1], [-1, 0, 1], [1, -1, 0]])
alpha = np.ones(m)
batch_size = 10**7 #2.5sec for 10**7 samples

batch_results = []

for i in range(len(batch_results)*batch_size, int(num_samples), batch_size):
    p_matr = rng.dirichlet(alpha, batch_size)
    samples = compute_lcs_volume_vectorized(p_matr, ssb_intr)
    batch_results.append((samples**2).mean())

with open(DATA_DIR / '5_1_MC_intransitive.dat', 'wb') as file:
    file.write(pack('d' * len(batch_results) , *batch_results))

print("Integral:", np.mean(batch_results), "std between batches (depnends on batch size):", np.std(batch_results))
print("Number of batches:", len(batch_results), "Probability of intransitivity: ", 1-3*np.mean(batch_results))

Integral: 0.2515930102536696 std between batches (depnends on batch size): 1.2659701519122817e-05
Number of batches: 7950 Probability of intransitivity:  0.24522096923899128


In [ ]:
num_samples = 2.65 * 10**10 #1 sample corresponds to 3 lotteries, a triple!
m = 3
rng = np.random.default_rng(2027)
ssb_intr = np.array([[0, 1, -1], [-1, 0, 1], [1, -1, 0]])
alpha = np.ones(m)
batch_size = 10**7 #2.0sec for 10**7 samples

batch_results = []

for i in range(len(batch_results)*batch_size, int(num_samples), batch_size):
    lotteries = rng.dirichlet(alpha, (3, batch_size))
    comp01 = (np.sum(lotteries[0,:,:] @ ssb_intr * lotteries[1,:,:], axis=1) > 0).astype(int)
    comp12 = (np.sum(lotteries[1,:,:] @ ssb_intr * lotteries[2,:,:], axis=1) > 0).astype(int)
    comp20 = (np.sum(lotteries[2,:,:] @ ssb_intr * lotteries[0,:,:], axis=1) > 0).astype(int)
    num_intr = np.sum(comp01 + comp12 + comp20 == 0) + np.sum(comp01 + comp12 + comp20 == 3)
    batch_results.append(num_intr/batch_size)

with open(DATA_DIR / '5_1_sim_intransitive.dat', 'wb') as file:
    file.write(pack('d' * len(batch_results) , *batch_results))

print(np.mean(batch_results), np.std(batch_results), len(batch_results))

0.24521831109433967 0.00013607908166060937 2650
